# Profile Python

https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html summary:

* `%time`: Time the execution of a single statement
* `%timeit`: Time repeated execution of a single statement for more accuracy
* `%prun`: Run code with the profiler
* `%lprun`: Run code with the line-by-line profiler
* `%memit`: Measure the memory use of a single statement
* `%mprun`: Run code with the line-by-line memory profiler

See also <BR>
http://codedelish.com/resources/Python/profiling.html<BR>
http://pynash.org/2013/03/06/timing-and-profiling/

In [1]:
import sys
print(sys.version)

3.6.6 | packaged by conda-forge | (default, Oct 12 2018, 14:08:43) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]


In [2]:
!pip install line-profiler
!pip install psutil
!pip install memory_profiler

two functions that we will profile

In [4]:
def mult_by(inpt):
    """
    a simple function that is not complicated
    """
    avalu = inpt*2
    avalu+=1
    return

def primes(n=1000): 
    """
    A more complicated function to be profiled
    
    An underdocumented function with an unclear purpose
    in-line comments added by Ben
    """
    A = [True] * (n+1) # the name of this array is not descriptive :(
    A[0] = False # there is no reasoning provided about why this would be 
    A[1] = False # no reasoning :(
    for i in range(2, int(n**0.5)): # I dislike single-letter variable names
        if A[i]: # again, this is difficult to reason about, both in terms of A and i
            mult_by(i) # not actually relevant to determining a prime number
            for j in range(i**2, n+1, i): # how did the author come up with this?
                A[j] = False

    return [x for x in range(2, n) if A[x]] # for me this is challenging to reason about

# analysis in three parts
1. total time
1. time per function
1. time per line in a specific function

# total time spent in execution using `time` magic function 

see https://ipython.readthedocs.io/en/stable/interactive/magics.html

How long does execution of a function take?

`%time` applies to a single line of code

In [5]:
primes(20) 

[2, 3, 5, 7, 11, 13, 17, 19]

In [6]:
%time primes(20) # call a function and provide an argument

CPU times: user 38 µs, sys: 10 µs, total: 48 µs
Wall time: 109 µs


[2, 3, 5, 7, 11, 13, 17, 19]

for "microseconds" (us, 1E-6) versus "milliseconds" (ms, 1E-3), see https://en.wikipedia.org/wiki/Metric_prefix#List_of_SI_prefixes

user versus system versus total: see https://en.wikipedia.org/wiki/CPU_time#Subdivision

We humans care about "wall time"; see https://en.wikipedia.org/wiki/Elapsed_real_time

<BR>
    
Similarly, `%%time` applies to the entire cell

In [7]:
%%time 
primes(20)
primes(30)

CPU times: user 1.99 ms, sys: 146 µs, total: 2.14 ms
Wall time: 2.16 ms


Although the two calls to `primes()` do output text, the display is overwritten by `time`

Displaying output skews measurement results for cell execution, so best practice is to store output to a variable

In [8]:
%time reslt=primes(20)

CPU times: user 41 µs, sys: 11 µs, total: 52 µs
Wall time: 63.9 µs


There's not much difference between times for multiple executions, but the variance increases when the duration is longer

In [9]:
%time reslt=primes(1000000)

print('length of array:',len(reslt))

CPU times: user 864 ms, sys: 85.2 ms, total: 949 ms
Wall time: 994 ms
length of array: 78498


To run multiple timing test, use `%timeit`

In [10]:
%timeit reslt=primes(1000000)

418 ms ± 70.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Want to capture the output of `timeit` to a variable? See https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module

# time spent per function using `cProfile`

https://docs.python.org/3/library/profile.html

In [11]:
import cProfile

In [12]:
cProfile.run('primes(10000000)')

         451 function calls in 7.515 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      446    0.001    0.000    0.001    0.000 <ipython-input-4-2fc7f0ed043d>:1(mult_by)
        1    1.912    1.912    1.912    1.912 <ipython-input-4-2fc7f0ed043d>:25(<listcomp>)
        1    5.425    5.425    7.338    7.338 <ipython-input-4-2fc7f0ed043d>:9(primes)
        1    0.176    0.176    7.515    7.515 <string>:1(<module>)
        1    0.000    0.000    7.515    7.515 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




* `ncalls`:       How many times it was called
* `tottime`:      for the total time spent in the given function (and excluding time made in calls to sub-functions)
* `percall`:      is the quotient of tottime divided by ncalls
* `cumtime`:      is the cumulative time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
* `percall`:      is the quotient of cumtime divided by primitive calls
* `filename:lineno(function)`:    provides the respective data of each function

# time spent per line within specified function

https://nyu-cds.github.io/python-performance-tuning/03-line_profiler/
    
https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

In [13]:
%load_ext line_profiler

In [14]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat

Profile the lines of the `prime` function

In [15]:
%lprun -f primes primes(10000)

Timer unit: 1e-06 s

Total time: 0.238713 s
File: <ipython-input-4-2fc7f0ed043d>
Function: primes at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def primes(n=1000): 
    10                                               """
    11                                               A more complicated function to be profiled
    12                                               
    13                                               An underdocumented function with an unclear purpose
    14                                               in-line comments added by Ben
    15                                               """
    16         1       1199.0   1199.0      0.5      A = [True] * (n+1) # the name of this array is not descriptive :(
    17         1          4.0      4.0      0.0      A[0] = False # there is no reasoning provided about why this would be 
    18         1          2.0      2.0      0.0      A[1] = Fal

profile `mult_by` function when `primes()` is executed

In [ ]:
%lprun -f mult_by primes(10000)

* `Line #`: The line number in the code
* `Hits`: The number of times that line was executed
* `Time`: The total amount of time spent executing the line in the timer’s units
* `Per Hit`: The average amount of time spent executing the line once in the timer’s unit
* `% Time`: The percentage of time spent on that line relative to the total amount of recorded time spent in the function
* `Line Contents`: The actual source code of the line

line_profiler generates profiles based on the lines in a program.

Line based profiling can provide more information about where a program is performing badly.